In [1]:
import numpy as np
import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import tensorflow as tf
from keras.models import Model
from keras.applications import mobilenet_v2
from keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight 
from tensorflow.python.keras.preprocessing import image as image_preprocessing

import random
from numpy.random import seed
seed(1)
from tensorflow.python.eager import context
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util

from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_preprocessing_layer
from tensorflow.python.keras.preprocessing import image as image_preprocessing
from tensorflow.python.keras.utils import control_flow_util
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from tensorflow import keras
#from tensorflow.keras.preprocessing.image import image_dataset_from_directory
train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   shear_range=5,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_ds = train_datagen.image_dataset_from_directory(
    directory='training',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
validation_ds = train_datagen.image_dataset_from_directory(
    directory='validation',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

In [7]:
train_input_shape = (256, 256, 3)

base_model=ResNet50(weights='imagenet', include_top=False, input_shape=train_input_shape) 

for layer in base_model.layers:
    layer.trainable = False    

x = base_model.output
x = Flatten()(x) 
x = Dense(100, activation='relu')(x)
x = Dense(50, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x) 


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
#model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
125/125 [==============================] - 383s 3s/step - loss: 4.1001 - accuracy: 0.4030 - val_loss: 1.9816 - val_accuracy: 0.4687
Epoch 2/10
125/125 [==============================] - 391s 3s/step - loss: 0.8616 - accuracy: 0.7460 - val_loss: 1.7853 - val_accuracy: 0.5707
Epoch 3/10
125/125 [==============================] - 1348s 11s/step - loss: 0.4235 - accuracy: 0.8653 - val_loss: 2.1048 - val_accuracy: 0.5414
Epoch 4/10
125/125 [==============================] - 378s 3s/step - loss: 0.1979 - accuracy: 0.9353 - val_loss: 2.0977 - val_accuracy: 0.5758
Epoch 5/10
125/125 [==============================] - 378s 3s/step - loss: 0.1272 - accuracy: 0.9574 - val_loss: 2.2173 - val_accuracy: 0.5889
Epoch 6/10
125/125 [==============================] - 380s 3s/step - loss: 0.0860 - accuracy: 0.9747 - val_loss: 2.3518 - val_accuracy: 0.5848
Epoch 7/10
125/125 [==============================] - 378s 3s/step - loss: 0.0745 - accuracy: 0.9777 - val_loss: 2.5421 - val_accuracy: 0.61

It seems to be overfitting ^

In [12]:
import os

directory_list = list()
for root, dirs, files in os.walk('training', topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

print(directory_list)

['training/Monet', 'training/VanGogh', 'training/Renoir', 'training/Matisse', 'training/Degas', 'training/Hassam', 'training/Sargent', 'training/Cezanne', 'training/Gauguin', 'training/Pissarro']


New model with batch normalization as a form of regularization

In [14]:
X = base_model.output
X = Flatten()(X)

X = Dense(512, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)
X = Dense(16, kernel_initializer='he_uniform')(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

predictions2 = Dense(10, activation='softmax')(X)

model2 = Model(inputs=base_model.input, outputs=predictions2)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model2.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
125/125 [==============================] - 405s 3s/step - loss: 1.6750 - accuracy: 0.4840 - val_loss: 1.2512 - val_accuracy: 0.6232
Epoch 2/10
125/125 [==============================] - 407s 3s/step - loss: 0.8215 - accuracy: 0.8576 - val_loss: 1.1370 - val_accuracy: 0.6939
Epoch 3/10
125/125 [==============================] - 411s 3s/step - loss: 0.3346 - accuracy: 0.9719 - val_loss: 1.0596 - val_accuracy: 0.7081
Epoch 4/10
125/125 [==============================] - 410s 3s/step - loss: 0.1327 - accuracy: 0.9937 - val_loss: 1.0029 - val_accuracy: 0.7071
Epoch 5/10
125/125 [==============================] - 431s 3s/step - loss: 0.0650 - accuracy: 0.9992 - val_loss: 0.9573 - val_accuracy: 0.7182
Epoch 6/10
125/125 [==============================] - 445s 4s/step - loss: 0.0377 - accuracy: 1.0000 - val_loss: 0.9434 - val_accuracy: 0.7242
Epoch 7/10
125/125 [==============================] - 434s 3s/step - loss: 0.0244 - accuracy: 1.0000 - val_loss: 0.9374 - val_accuracy: 0.7212

In [20]:
X = base_model.output
X = Flatten()(X)

X = Dense(500, kernel_initializer='he_uniform')(X)
X = Dropout(0.2)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(20, kernel_initializer='he_uniform')(X)
X = Dropout(0.2)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

predictions3 = Dense(10, activation='softmax')(X)

model3 = Model(inputs=base_model.input, outputs=predictions3)
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model3.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
125/125 [==============================] - 420s 3s/step - loss: 1.7313 - accuracy: 0.4488 - val_loss: 1.1996 - val_accuracy: 0.6384
Epoch 2/10
125/125 [==============================] - 423s 3s/step - loss: 1.0065 - accuracy: 0.7771 - val_loss: 1.1049 - val_accuracy: 0.6798
Epoch 3/10
125/125 [==============================] - 449s 4s/step - loss: 0.5509 - accuracy: 0.9050 - val_loss: 1.0565 - val_accuracy: 0.6970
Epoch 4/10
125/125 [==============================] - 441s 4s/step - loss: 0.3102 - accuracy: 0.9639 - val_loss: 1.0079 - val_accuracy: 0.6949
Epoch 5/10
125/125 [==============================] - 1845s 15s/step - loss: 0.1936 - accuracy: 0.9789 - val_loss: 0.9717 - val_accuracy: 0.7111
Epoch 6/10
125/125 [==============================] - 2557s 21s/step - loss: 0.1220 - accuracy: 0.9935 - val_loss: 0.9397 - val_accuracy: 0.7212
Epoch 7/10
125/125 [==============================] - 608s 5s/step - loss: 0.0932 - accuracy: 0.9942 - val_loss: 0.9181 - val_accuracy: 0.

In [28]:
X = base_model.output
X = Flatten()(X)

X = Dense(500, kernel_initializer='he_uniform')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(100, kernel_initializer='he_uniform')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

predictions4 = Dense(10, activation='softmax')(X)

model4 = Model(inputs=base_model.input, outputs=predictions4)
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model4.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
125/125 [==============================] - 416s 3s/step - loss: 1.4919 - accuracy: 0.5203 - val_loss: 1.0354 - val_accuracy: 0.6697
Epoch 2/10
125/125 [==============================] - 415s 3s/step - loss: 0.6436 - accuracy: 0.8242 - val_loss: 0.9953 - val_accuracy: 0.6859
Epoch 3/10
125/125 [==============================] - 432s 3s/step - loss: 0.2608 - accuracy: 0.9473 - val_loss: 0.9518 - val_accuracy: 0.7152
Epoch 4/10
125/125 [==============================] - 452s 4s/step - loss: 0.1050 - accuracy: 0.9872 - val_loss: 0.9280 - val_accuracy: 0.7051
Epoch 5/10
125/125 [==============================] - 443s 4s/step - loss: 0.0584 - accuracy: 0.9932 - val_loss: 0.9174 - val_accuracy: 0.7152
Epoch 6/10
125/125 [==============================] - 403s 3s/step - loss: 0.0344 - accuracy: 0.9970 - val_loss: 0.8960 - val_accuracy: 0.7283
Epoch 7/10
125/125 [==============================] - 397s 3s/step - loss: 0.0228 - accuracy: 0.9987 - val_loss: 0.9034 - val_accuracy: 0.7232

In [29]:
history=model4.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
125/125 [==============================] - 449s 4s/step - loss: 1.4680 - accuracy: 0.5226 - val_loss: 1.0829 - val_accuracy: 0.6556
Epoch 2/10
125/125 [==============================] - 492s 4s/step - loss: 0.6388 - accuracy: 0.8195 - val_loss: 0.9323 - val_accuracy: 0.6990
Epoch 3/10
125/125 [==============================] - 493s 4s/step - loss: 0.2456 - accuracy: 0.9519 - val_loss: 0.9046 - val_accuracy: 0.7101
Epoch 4/10
125/125 [==============================] - 490s 4s/step - loss: 0.1116 - accuracy: 0.9852 - val_loss: 0.8843 - val_accuracy: 0.7192
Epoch 5/10
125/125 [==============================] - 492s 4s/step - loss: 0.0588 - accuracy: 0.9947 - val_loss: 0.8978 - val_accuracy: 0.7131
Epoch 6/10
125/125 [==============================] - 523s 4s/step - loss: 0.0396 - accuracy: 0.9962 - val_loss: 0.9133 - val_accuracy: 0.7192
Epoch 7/10
125/125 [==============================] - 554s 4s/step - loss: 0.0256 - accuracy: 0.9975 - val_loss: 0.9566 - val_accuracy: 0.7040

try: image augmentation: ImageDataGenerator from keras 

make confusion matrix

In [30]:
X = base_model.output
X = Flatten()(X)

X = Dense(100, kernel_initializer='he_uniform')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(50, kernel_initializer='he_uniform')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

predictions5 = Dense(10, activation='softmax')(X)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
model5 = Model(inputs=base_model.input, outputs=predictions5)
model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
history_model5=model5.fit(train_ds,epochs=20,validation_data=validation_ds,callbacks=[reduce_lr])

Epoch 1/20
125/125 [==============================] - 420s 3s/step - loss: 1.8236 - accuracy: 0.3902 - val_loss: 1.2502 - val_accuracy: 0.6061
Epoch 2/20
125/125 [==============================] - 446s 4s/step - loss: 1.0924 - accuracy: 0.6843 - val_loss: 1.1040 - val_accuracy: 0.6667
Epoch 3/20
125/125 [==============================] - 462s 4s/step - loss: 0.6596 - accuracy: 0.8370 - val_loss: 0.9904 - val_accuracy: 0.6869
Epoch 4/20
125/125 [==============================] - 466s 4s/step - loss: 0.3924 - accuracy: 0.9085 - val_loss: 0.9932 - val_accuracy: 0.6798
Epoch 5/20
125/125 [==============================] - 490s 4s/step - loss: 0.2594 - accuracy: 0.9491 - val_loss: 0.9529 - val_accuracy: 0.7121
Epoch 6/20
125/125 [==============================] - 479s 4s/step - loss: 0.1752 - accuracy: 0.9646 - val_loss: 1.0051 - val_accuracy: 0.6939
Epoch 7/20
125/125 [==============================] - 455s 4s/step - loss: 0.1229 - accuracy: 0.9782 - val_loss: 0.9438 - val_accuracy: 0.7111

In [98]:
X = base_model.output
X = Flatten()(X)

X = Dense(100, kernel_initializer='random_normal')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(50, kernel_initializer='random_normal')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

predictions6 = Dense(10, activation='softmax')(X)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=8, 
    min_delta=0.001, 
    mode='max',
    restore_best_weights=True
)

opt = keras.optimizers.Adam(learning_rate=0.001)

model6 = Model(inputs=base_model.input, outputs=predictions6)
model6.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [99]:
history_model6=model6.fit(train_ds,epochs=15,validation_data=validation_ds,callbacks=[custom_early_stopping,reduce_lr])

Epoch 1/15
125/125 [==============================] - 1056s 8s/step - loss: 1.7511 - accuracy: 0.4230 - val_loss: 1.2544 - val_accuracy: 0.5939
Epoch 2/15
125/125 [==============================] - 652s 5s/step - loss: 1.0064 - accuracy: 0.7106 - val_loss: 1.1137 - val_accuracy: 0.6434
Epoch 3/15
125/125 [==============================] - 710s 6s/step - loss: 0.5755 - accuracy: 0.8543 - val_loss: 0.9331 - val_accuracy: 0.7030
Epoch 4/15
125/125 [==============================] - 738s 6s/step - loss: 0.3309 - accuracy: 0.9330 - val_loss: 0.9436 - val_accuracy: 0.6990
Epoch 5/15
125/125 [==============================] - 761s 6s/step - loss: 0.1946 - accuracy: 0.9649 - val_loss: 1.0252 - val_accuracy: 0.6768


In [74]:
batch_size = 16
train_input_shape = (256, 256, 3)
n_classes = 10
images_dir='training'

train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255.,
                                   shear_range=5,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

test_set = train_datagen.flow_from_directory('training',target_size=(226,226),batch_size=16,class_mode='categorical')


Found 3988 images belonging to 10 classes.


Testing the model:

In [219]:
import imageio
import cv2
labels = test_set.class_indices
labels = dict((v,k) for k,v in labels.items())
img='testfolder/test1.jpg'

img = imageio.imread(img)
img = cv2.resize(img, dsize=train_input_shape[0:2], )
img = image.img_to_array(img)
img /= 255.
img = np.expand_dims(img, axis=0)
p = model6.predict(img)
print("predictions: ",p)
prob = np.amax(p)
idx = np.argmax(p)
print('preds:',p,labels[idx])

print("Artist: ", labels[idx].replace('_', ' '), "(probability: {:.2f}%)".format(prob*100))


predictions:  [[0.07220162 0.09561805 0.10533102 0.0970286  0.0554305  0.16482826
  0.18173131 0.10521411 0.04832559 0.0742909 ]]
preds: [[0.07220162 0.09561805 0.10533102 0.0970286  0.0554305  0.16482826
  0.18173131 0.10521411 0.04832559 0.0742909 ]] Pissarro
Artist:  Pissarro (probability: 18.17%)


In [119]:
model6.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Let's turn this into a binary classification problem to see if it improves: (note: after testing the binary model, there is still a bias problem)

In [145]:
train_ds_bin = image_dataset_from_directory(
    directory='training2',
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256))
validation_ds_bin = image_dataset_from_directory(
    directory='validation2',
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256))

Found 798 files belonging to 2 classes.
Found 198 files belonging to 2 classes.


In [162]:
X = base_model.output
X = Flatten()(X)

X = Dense(400, activation='relu')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

X = Dense(50, activation='relu')(X)
X = Dropout(0.3)(X)
X = BatchNormalization()(X)
X = Activation('relu')(X)

bin_predictions = Dense(2, activation='sigmoid')(X)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=8, 
    min_delta=0.001, 
    mode='max',
    restore_best_weights=True
)

opt = keras.optimizers.Adam(learning_rate=0.001)

bin_model = Model(inputs=base_model.input, outputs=bin_predictions)
bin_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
'''history_bin_model=bin_model.fit(train_ds_bin,epochs=15,validation_data=validation_ds_bin,
                          callbacks=[custom_early_stopping,reduce_lr])'''

In [171]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

In [173]:
import sklearn.metrics as metrics

report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

              precision    recall  f1-score   support

     Cezanne       0.96      0.89      0.93       399
       Degas       0.90      0.97      0.93       398
     Gauguin       0.97      0.89      0.93       399
      Hassam       0.86      0.99      0.92       399
     Matisse       0.96      0.92      0.94       399
       Monet       0.84      0.95      0.89       399
    Pissarro       0.96      0.89      0.92       398
      Renoir       0.91      0.96      0.93       399
     Sargent       0.99      0.69      0.81       399
     VanGogh       0.86      0.97      0.91       399

    accuracy                           0.91      3988
   macro avg       0.92      0.91      0.91      3988
weighted avg       0.92      0.91      0.91      3988

